In [ ]:
 # Self Organizing Map

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Credit_Card_Applications.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X)

# Training the SOM
from minisom import MiniSom

#input_len=no of columns
#sigma=radius of circle
som = MiniSom(x = 10, y = 10, input_len = 15, sigma = 1.0, learning_rate = 0.5)
som.random_weights_init(X)
som.train_random(data = X, num_iteration = 100)
#we take minimum distance

# Visualizing the results
from pylab import bone, pcolor, colorbar, plot, show
#initialize the figure
bone()
#calculate normalized distance between winning neuron and input vectors

pcolor(som.distance_map().T)
#legend
colorbar()
# 'o' -circle   's' -square
#customer that are non fraudlent are having low mid ie are black whereas whte ones are fraudlent
markers = ['o', 's']
colors = ['r', 'g']
#w contains the coordinates
#circle who doesn't got approval
#square who got approval
#color denote whether they are fraulent or not
for i, x in enumerate(X):
    w = som.winner(x)
    plot(w[0] + 0.5,
         w[1] + 0.5,
         markers[y[i]],
         markeredgecolor = colors[y[i]],
         markerfacecolor = 'None',
         markersize = 10,
         markeredgewidth = 2)
show()

# Finding the frauds
mappings = som.win_map(X)
#mapping dict key is coordinates in map list with all the patterns that have been mapped in the position 

#find the coordinates to who are most likely to be fraudlent ie white ones
frauds = np.concatenate((mappings[(5,5)], mappings[(7,3)]), axis = 0)
frauds = sc.inverse_transform(frauds)

#white ones are fraudlents and further study will be done 

customers = dataset.iloc[:, 1:].values
is_fraud=np.zeros(len(dataset))
for i in range(len(dataset)):
    if dataset.iloc[i,0] in frauds:
        is_fraud[i]=1
        
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
customers=sc.fit_transform(customers)        
        
from keras.layers import Dense
from keras.models import Sequential

classifier=Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'relu', input_dim = 15))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(customers,is_fraud, batch_size = 1, epochs = 2)
y_pred=classifier.predict(customers)
y_pred=np.concatenate((dataset.iloc[:,0:1].values,y_pred),axis=1)

y_pred=y_pred[y_pred[:,1].argsort()]
        
        